# Distance Computation

Non-functional

A script to compute the distance from each hand-joint to the object, based on the object meshes.

Output:
- Distance map for each sequence: NDArray (N, 42) for N frames per sequence
  - 21 (left hand) + 21 (right hand) distances from joints

----

Imports

In [2]:
import sys
sys.path.append('..')
import os
import numpy as np
from numpy.typing import NDArray
import open3d as o3d
import tqdm
from typing import List
import utils.transformations as tf
import utils.data_format as data_format

Parameters

In [21]:
n_frames_per_seq : int = 8
n_point_samples : int = 100
mode = 'val' # 'train' , 'val' , 'test'

h2o_root = '../data/h2o/'
sample_root = h2o_root + f'seq_{n_frames_per_seq}_{mode}/'

## Script

Helpers

In [22]:
obj_mesh_files = {
    0: '1_book.ply',
    1: '2_espresso.ply',
    2: '3_lotion.ply',
    3: '4_spray.ply',
    4: '5_milk.ply',
    5: '6_cocoa.ply',
    6: '7_chips.ply',
    7: '8_cappuccino.ply'
}

def sample_points(n_samples: int, obj_mesh_dir: str) -> NDArray:
    mesh_samples = []
    for i in range(8):
        mesh = o3d.io.read_triangle_mesh(obj_mesh_dir + obj_mesh_files[i])
        pc = mesh.sample_points_poisson_disk(number_of_points=n_samples, init_factor=10)
        mesh_samples.append(np.asarray(pc.points))
    mesh_samples = np.array(mesh_samples)
    return mesh_samples


In [23]:
def compute_distance(hand_poses: NDArray, obj_samples: NDArray, T_CO: NDArray) -> NDArray:
    assert len(obj_samples.shape) == 2 and obj_samples.shape[1] == 3, \
        f"obj_samples should be of shape (N, 3) but got {obj_samples.shape}"
    assert len(hand_poses.shape) == 2 and hand_poses.shape[1] == 3, \
        f"hand_poses should be of shape (N, 3) but got {hand_poses.shape}"
    assert T_CO.shape == (4, 4), f"T_CO should be of shape (4, 4) but got {T_CO.shape}"
    
    # transform mesh points
    obj_samples = tf.transfrom_pts_cart(obj_samples, T_CO)

    # compute distance from every joint to all object samples
    n_joints = hand_poses.shape[0]
    n_samples = obj_samples.shape[0]
    distances = np.zeros((n_joints, n_samples))
    for i in range(n_joints):
        joint = hand_poses[i]
        distances[i] = np.linalg.norm(obj_samples - joint, axis=1)
    
    # compute minimal distance for each joint
    return np.min(distances, axis=1)

Preparations

In [24]:
# Load number of sequences for selected mode
sample_ids = np.load(sample_root + 'sample_ids.npy')
n_sequences = sample_ids.shape[0]
print(f"Loaded {n_sequences} action sequences for {mode}.")
# load the object classes for all sequences
obj_classes = np.load(sample_root + f'obj_classes_{mode}.npy')
assert obj_classes.shape == (n_sequences,)

# path handling
hand_dir = sample_root + f'poses_hand_{mode}/'
obj_rt_dir = sample_root + f'obj_rt_{mode}/'
obj_mesh_dir = h2o_root + 'object_meshes/'
dist_dir = sample_root + f'distances_{mode}/'
os.makedirs(dist_dir, exist_ok=True) # create the destination directory


Loaded 122 action sequences for val.


In [25]:
# sample 100 points on every mesh
obj_samples = sample_points(n_point_samples, obj_mesh_dir)
print("Sampled points on object meshes. Shape:", obj_samples.shape)

Sampled points on object meshes. Shape: (8, 100, 3)


In [26]:
for seq_id in range(1, n_sequences+1):
    distance_sequence = []

    # get poses of current sequence
    hand_poses = np.load(hand_dir + f'{seq_id:03d}.npy')
    obj_rts = np.load(obj_rt_dir + f'{seq_id:03d}.npy')
    assert hand_poses.shape == (n_frames_per_seq, 128)
    assert obj_rts.shape == (n_frames_per_seq, 4, 4)

    obj_class = obj_classes[seq_id-1] # seq_id is 1-indexed

    for frame_id in range(n_frames_per_seq):
        hand_left, hand_right = data_format.format_hand_poses(hand_poses[frame_id])
        joints = np.concatenate((hand_left, hand_right), axis=0)
        T_CO = obj_rts[frame_id]
        
        distance_sequence.append(compute_distance(joints, obj_samples[obj_class-1], T_CO))
    
    distance_sequence = np.array(distance_sequence)
    np.save(dist_dir + f'{seq_id:03d}.npy', distance_sequence)

print(f"Successfully computed the distances for {seq_id} sequences.")

Successfully computed the distances for 122 sequences.
